In [1]:
from cargar_datos import get_mourning_df

from gensim.utils import any2unicode as unicode
from nltk.stem import SnowballStemmer
from sklearn.utils import resample
import os, re, sys, pandas, unidecode, string

import numpy
import emoji
from nltk.tokenize import TweetTokenizer

In [2]:
df = get_mourning_df(1, 0)
df

Preparando df 100.0%
Normalizando df 100.0%
Creando sellos de balanceamiento 99.21%

In [3]:
df2 = get_mourning_df(1, 1)
df2

Preparando df 100.0%
Normalizando df 100.0%
Creando sellos de balanceamiento 100.0%
Balanceando df
Df mourning entregado



,text,lang,mourning
0,"@nabybi @_cinco8 Cuentos de la cripta, o la Ve...",es,0
1,También os digo que yo ahí era todavía hetero ...,es,0
2,"""No la menciones a ella porfavor"". JAJAJAJA",es,0
3,@L_alvarezS El lado positivo es que no es un N...,es,0
4,"@vicayala Buena cantaleta se han inventado , e...",es,0
...,...,...,...
16695,Vancouver mayor Kennedy Stewart is making his ...,en,1
16696,In memory of a life so beautifully lived and a...,en,1
16697,Today there were 27 new cases of #COVID19 in #...,en,1
16698,"Damn, COVID got the double rainbow guy. RIP.",en,1


In [4]:
df_en = df[df['lang']=="en"]
df_es = df[df['lang']=="es"]

In [6]:
def tokenizar(text):
    tweet_tokenizer = TweetTokenizer()
    
    text = re.sub(r'[\b@]\w+\s{1}', '', text)#Quitar menciones
    words = tweet_tokenizer.tokenize(text)
    palabras = []
    for w in words:
        if re.match(r'\W', w)==None:
            p = unidecode.unidecode(w).lower() #Quitar Acentos minuscula
            p = re.sub(r'\b(?=\w*[j])[aeiouj]{2,}\b', 'jajaja', p) #Normalizar Risa
            p = re.sub(r'\b(?=\w*[h])[aeiouh]{2,}\b', 'jajaja', p) #Normalizar Risa
            palabras.append(p)
                
        elif re.match(r'#', w)!=None:
            palabras.append(unidecode.unidecode(w).lower())
        elif  w in emoji.UNICODE_EMOJI:
            palabras.append(w)
    return palabras


In [7]:
def lexicon_mornig(df):
    
    print()
    
    lexicon = dict()
    morning = [[],[]] # [no morning, morning]
    
    cont=0
    
    cont_morning = dict()
    cont_no_morning = dict()
    cont_total = dict()
    
    vocabulario = set([])
    
    i=0
    for index, row in df.iterrows():
        i+=1
        sys.stdout.write("\rCreanco lexicon df " + str(round(((i) / (df.shape[0])) * 100, 2)) + "%")
        sys.stdout.flush()
        
        cont += 1
                
        palabras = tokenizar(row['text'])  
        
        vocabulario|=set(palabras)
        morning[int(row['mourning'])].append(set(palabras))    
        #print(row['text'])
        #print(palabras)
        
    for v in morning[1]:
        for m in v:
            cont_morning[m]=1 if m not in cont_morning else cont_morning[m]+1
            cont_total[m]=1 if m not in cont_total else cont_total[m]+1
            
    for v in morning[0]:
        for m in v:
            cont_no_morning[m]=1 if m not in cont_morning else cont_morning[m]+1
            cont_total[m]=1 if m not in cont_total else cont_total[m]+1
            
    for w in vocabulario:
        if cont_total[w] > 5: 
            #if w not in cont_morning:
            #    lexicon[w] = 0#-(cont_no_morning[w]/len(morning[0]))*(cont_total[w]/cont) 
            #else:
            #    lexicon[w]=(cont_morning[w]/len(morning[1]))*(cont_total[w]/cont)
            lexicon[w]=0
            if w in cont_no_morning:
                lexicon[w] -= (cont_no_morning[w]/len(morning[0]))*(cont_total[w]/cont) 
                
            if w in cont_morning:
                lexicon[w] += (cont_morning[w]/len(morning[1]))*(cont_total[w]/cont) 
    return lexicon



In [8]:
def vectorizar(df, lexicon):
    
    print()
    
    keys = tuple(lexicon.keys())
    
    y = dict()
    
    for a in keys:
        y[a]=list(numpy.zeros(df.shape[0]))
        
    i=0
    for index, row in df.iterrows():
        
        sys.stdout.write("\rVectorizando df " + str(round(((i+1) / (df.shape[0])) * 100, 2)) + "%")
        sys.stdout.flush()
        
        v = dict()
        
        palabras = tokenizar(row['text']) 
        for w in palabras:
            if w in lexicon:
                y[w][i]=lexicon[w]
        i+=1
        
    vector = pandas.DataFrame.from_dict(y)
        
        
    return vector


In [9]:
def vectorizar3(df, lexicon):
    
    print()
    
    keys = tuple(lexicon.keys())
    
    y = dict()
    
    for a in keys:
        y[a]=list(numpy.zeros(df.shape[0]))
    y[1]=list(numpy.zeros(df.shape[0]))
        
    i=0
    for index, row in df.iterrows():
        
        sys.stdout.write("\rVectorizando df " + str(round(((i+1) / (df.shape[0])) * 100, 2)) + "%")
        sys.stdout.flush()
        
        v = dict()
        palabras = tokenizar(row['text']) 
        for w in palabras:
            if w in lexicon:
                y[w][i]=lexicon[w]
                y[1][i]+=lexicon[w]
        i+=1
        
    vector = pandas.DataFrame.from_dict(y)
        
        
    return vector


In [ ]:
def vectorizar2(df, lexicon):
    
    print()
    
    keys = tuple(lexicon.keys())
    
    y = {'valor':[]}
       
    i=0
    for index, row in df.iterrows():
        
        suma=0
        
        sys.stdout.write("\rVectorizando df " + str(round(((i+1) / (df.shape[0])) * 100, 2)) + "%")
        sys.stdout.flush()
        
        v = dict()
        
        palabras = tokenizar(row['text']) 
        for w in palabras:
            if w in lexicon:
                suma+=lexicon[w]
        y['valor'].append(suma)
        i+=1
        
    vector = pandas.DataFrame.from_dict(y)
        
        
    return vector


In [10]:
a = df_en
l = lexicon_mornig(a)
values = vectorizar3(a, l).values


Creanco lexicon df 100.0%
Vectorizando df 100.0%

In [11]:
from sklearn.cluster import KMeans
real = list(a['mourning'])

model = KMeans(n_clusters=2)
model.fit(values)
labels = model.fit_predict(values)
df_comparison = pandas.DataFrame({'labels': labels, 'mourning': real})
df_comparison

,labels,mourning
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0
...,...,...
8345,1,1
8346,1,1
8347,1,1
8348,0,1


In [12]:
tabla_pivot = pandas.crosstab(df_comparison['labels'], df_comparison['mourning'])
tabla_pivot

mourning,0,1
labels,,
0,2966,2116
1,1209,2059


In [13]:
from sklearn.cluster import KMeans

#a= pandas.concat([df[df['mourning']=="1"][:1000], df[df['mourning']=="0"][:1000]])
a = df_es
l = lexicon_mornig(a)
values = vectorizar(a, l).values

real = list(a['mourning'])

model = KMeans(n_clusters=2)
model.fit(values)
labels = model.fit_predict(values)
df_comparison = pandas.DataFrame({'labels': labels, 'mourning': real})
df_comparison


Creanco lexicon df 100.0%
Vectorizando df 100.0%

,labels,mourning
0,1,0
1,0,0
2,0,0
3,0,0
4,1,0
...,...,...
8345,1,1
8346,1,1
8347,1,1
8348,1,1


In [14]:
tabla_pivot = pandas.crosstab(df_comparison['labels'], df_comparison['mourning'])
tabla_pivot

mourning,0,1
labels,,
0,2078,1685
1,2097,2490


In [ ]:
from sklearn.mixture import GaussianMixture

#a= pandas.concat([df[df['mourning']=="1"][:1000], df[df['mourning']=="0"][:1000]])
a = df_en
l = lexicon_mornig(a)
values = vectorizar(a, l).values

real = list(a['mourning'])

model = GaussianMixture(n_components=2)
model.fit(values)
labels = model.fit_predict(values)
df_comparison = pandas.DataFrame({'labels': labels, 'mourning': real})
df_comparison

In [ ]:
tabla_pivot = pandas.crosstab(df_comparison['labels'], df_comparison['mourning'])
tabla_pivot